FINAL API

In [94]:
def parse_cve_data(cve_items):
    # cve_items = data.get("result", {}).get("CVE_Items", [])
    parsed_data = []

    for item in cve_items:
        cve_id = item.get("cve", {}).get("CVE_data_meta", {}).get("ID", "N/A")
        description = item.get("cve", {}).get("description", {}).get("description_data", [{}])[0].get("value", "No description available")
        
        base_score = item.get("impact", {}).get("baseMetricV3", {}).get("cvssV3", {}).get("baseScore", "N/A")
        base_severity = item.get("impact", {}).get("baseMetricV3", {}).get("cvssV3", {}).get("baseSeverity", "N/A")
        combined_score = f"{base_score} {base_severity}" if base_score != "N/A" else "N/A"

        published_date_iso = item.get("publishedDate", "N/A")
        if published_date_iso != "N/A":
            published_date_obj = datetime.strptime(published_date_iso, "%Y-%m-%dT%H:%MZ")
            published_date = published_date_obj.strftime("%d/%m/%Y")
        else:
            published_date = "N/A"

        references = [ref.get("url") for ref in item.get("cve", {}).get("references", {}).get("reference_data", [])]

        parsed_data.append({
            "CVE ID": cve_id,
            "Description": description,
            "Base Score": combined_score,
            "Published Date": published_date,
            "References": references
        })

    return parsed_data


In [95]:
import json

def save_to_json(data, filename):
    """Saves data to a JSON file."""
    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)

In [96]:
import requests
from datetime import datetime, timedelta
from dotenv import load_dotenv
import os
import time

# Load environment variables
load_dotenv()

def format_date(date):
    return date.strftime("%Y-%m-%dT%H:%M:%S:000%%20UTC-05:00")

def fetch_cves(api_key, search_term, resultsPerPage, max_results, start_date, end_date):
    base_url = "https://services.nvd.nist.gov/rest/json/cves/1.0"
    results = []
    current_end_date = end_date

    while len(results) < max_results and current_end_date > start_date:
        current_start_date = max(current_end_date - timedelta(days=120), start_date)
        formatted_start_date = format_date(current_start_date)
        formatted_end_date = format_date(current_end_date)

        url = (f"{base_url}?apiKey={api_key}&keyword={search_term}&"
               f"resultsPerPage={resultsPerPage}&sortBy=publishDate&"
               f"pubStartDate={formatted_start_date}&pubEndDate={formatted_end_date}")

        print(f"Fetching data from: {url}")  # Debugging: print the URL being requested

        try:
            response = requests.get(url)
            response.raise_for_status()
            data = response.json()
            cve_items = data.get("result", {}).get("CVE_Items", [])
            results.extend(cve_items)

            print(f"Found {len(cve_items)} items")  # Debugging: print the number of items found

            if len(cve_items) < 2000:
                current_end_date = current_start_date  # Move to the previous period
            else:
                # Handle pagination if necessary
                pass
        except requests.exceptions.RequestException as e:
            print(f"Error fetching data: {e}")
            break

        time.sleep(8)  # Delay for 8 seconds

        if len(results) >= max_results:
            results = results[:max_results]
            break

    return results


# Output
api_key = os.getenv("NVD_API_KEY")
search_terms = ["Android", "iOS", "Windows", "networking"]  # Example search terms
start_date = datetime(2021, 12, 17)
end_date = datetime(2023, 12, 17)
filename = "combined_cve_data.json"

combined_results = []

for term in search_terms:
    cve_data = fetch_cves(api_key, term, 2000, 3000, start_date, end_date)
    parsed_data = parse_cve_data(cve_data)
    combined_results.extend(parsed_data)

    print(f"Fetched {len(parsed_data)} CVEs for {term}")
    print("\n")  # Blank line after each search term

# Save the combined results to a JSON file
save_to_json(combined_results, filename)
print(f"Total CVEs fetched: {len(combined_results)}")
print(f"Results saved to {filename}")

Fetching data from: https://services.nvd.nist.gov/rest/json/cves/1.0?apiKey=c9c7405e-a7a4-47b7-8e34-9991daa1a80e&keyword=Android&resultsPerPage=2000&sortBy=publishDate&pubStartDate=2023-08-19T00:00:00:000%20UTC-05:00&pubEndDate=2023-12-17T00:00:00:000%20UTC-05:00
Found 32 items
Fetching data from: https://services.nvd.nist.gov/rest/json/cves/1.0?apiKey=c9c7405e-a7a4-47b7-8e34-9991daa1a80e&keyword=Android&resultsPerPage=2000&sortBy=publishDate&pubStartDate=2023-04-21T00:00:00:000%20UTC-05:00&pubEndDate=2023-08-19T00:00:00:000%20UTC-05:00
Found 226 items
Fetching data from: https://services.nvd.nist.gov/rest/json/cves/1.0?apiKey=c9c7405e-a7a4-47b7-8e34-9991daa1a80e&keyword=Android&resultsPerPage=2000&sortBy=publishDate&pubStartDate=2022-12-22T00:00:00:000%20UTC-05:00&pubEndDate=2023-04-21T00:00:00:000%20UTC-05:00
Found 294 items
Fetching data from: https://services.nvd.nist.gov/rest/json/cves/1.0?apiKey=c9c7405e-a7a4-47b7-8e34-9991daa1a80e&keyword=Android&resultsPerPage=2000&sortBy=publi

DESERIALIZE DESCRIPTIONS

In [2]:
import json

def clean_description(description):
    #Converts specific escape sequences in description.
    return description.replace('\n', '').replace('\r', '')

def print_items_from_json(filename):
    #Reads a JSON file and prints each item with a cleaned description.
    with open(filename, 'r', encoding='utf-8') as file:
        data = json.load(file)

        for item in data:
            # Clean the description
            item['Description'] = clean_description(item['Description'])

            # Print the item details
            print("CVE ID:", item.get("CVE ID", "N/A"))
            print("Description:", item.get("Description", "No description available"))
            print("Base Score:", item.get("Base Score", "N/A"))
            print("Published Date:", item.get("Published Date", "N/A"))
            print("References:", item.get("References", []))
            print("\n---\n")  # Separator between items

# Output
filename = "combined_cve_data.json"
print_items_from_json(filename)


CVE ID: CVE-2023-36621
Description: An issue was discovered in the Boomerang Parental Control application through 13.83 for Android. The child can use Safe Mode to remove all restrictions temporarily or uninstall the application without the parents noticing.
Base Score: 9.1 CRITICAL
Published Date: 03/11/2023
References: ['https://seclists.org/fulldisclosure/2023/Jul/12', 'https://sec-consult.com/blog/detail/the-hidden-costs-of-parental-control-apps/', 'https://useboomerang.com/']

---

CVE ID: CVE-2023-36620
Description: An issue was discovered in the Boomerang Parental Control application before 13.83 for Android. The app is missing the android:allowBackup="false" attribute in the manifest. This allows the user to backup the internal memory of the app to a PC. This gives the user access to the API token that is used to authenticate requests to the API.
Base Score: 4.6 MEDIUM
Published Date: 03/11/2023
References: ['https://seclists.org/fulldisclosure/2023/Jul/12', 'https://sec-consul